In [46]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import numpy as np
import string
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import random
from sklearn.utils import shuffle
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.metrics.pairwise import cosine_similarity

def stemming_tokenizer(text):
    stemmer = PorterStemmer()
    temp = text.replace("\\r", " ")
    temp = temp.replace("\\n", " ")
    return [stemmer.stem(w) for w in word_tokenize(temp)]

random.seed(42)

stop_words = stopwords.words('english')+list(string.punctuation)
stemmed_stop_words = stemming_tokenizer(" ".join(stop_words))

In [6]:
import gensim
# https://github.com/alexandres/lexvec - https://www.dropbox.com/s/flh1fjynqvdsj4p/lexvec.commoncrawl.300d.W.pos.vectors.gz?dl=1
model = gensim.models.KeyedVectors.load_word2vec_format('lexvec.enwiki+newscrawl.300d.W.pos.vectors', binary=False)

In [34]:
def get_vector(text, debug=False):
    vec = np.zeros(300)
    found_count = 0
    total_count = 0
    for word in stemming_tokenizer(text):
        if word.lower() in stemmed_stop_words:
            continue
        total_count += 1
        if word.lower() in model:
            vec = vec + model[word.lower()]
            found_count += 1
    if debug:
        print(found_count, total_count)
    return vec

In [31]:
# load data
data = pd.read_csv('miniature.csv')

0    help wanted,insiders-released,markdown,verific...
1    bug,candidate,custom-editors,insiders-released...
2                    bug,candidate,extensions,verified
3                       Product-Launcher,Triage-Needed
4    Issue-Bug,Product-Color Picker,Resolution-Fix-...
Name: Label, dtype: object

In [36]:
# Create vectors
X_issue_title_wordvec = [get_vector(text, True) for text in data['IssueTitle']]
X_issue_description_wordvec = [get_vector(text, True) for text in data['IssueDescription']]
X_issue_label_wordvec = [get_vector(text, True) for text in data['Label']]
Y_pr_title_wordvec = [get_vector(text, True) for text in data['PrTitle']]
Y_pr_description_wordvec = [get_vector(text, True) for text in data['PrDescription']]

5 5
3 5
1 3
4 6
7 8
37 69
76 131
74 129
34 53
13 16
3 5
5 7
3 4
0 2
1 4
6 7
3 6
4 7
3 3
7 7
40 54
6 13
6 10
104 144
85 115


In [38]:
# Train-test split for all the data
X_issue_title_wordvec_train, X_issue_title_wordvec_test, X_issue_description_wordvec_train, X_issue_description_wordvec_test, X_issue_label_wordvec_train, X_issue_label_wordvec_test, Y_pr_title_wordvec_train, Y_pr_title_wordvec_test, Y_pr_description_wordvec_train, Y_pr_description_wordvec_test = train_test_split(X_issue_title_wordvec, X_issue_description_wordvec, X_issue_label_wordvec, Y_pr_title_wordvec, Y_pr_description_wordvec, test_size=0.25, random_state=33)

In [42]:
# Model
classifier = MLPRegressor(hidden_layer_sizes=(100))
classifier.fit(X_issue_description_wordvec_train, Y_pr_description_wordvec_train)

MLPRegressor(hidden_layer_sizes=100)

In [49]:
# Prediction
y_pred = classifier.predict(X_issue_description_wordvec_train)
cosine_similarity(y_pred, Y_pr_description_wordvec_train)

array([[0.99995536, 0.65876328, 0.64161949],
       [0.6586168 , 0.99998496, 0.94782438],
       [0.64182454, 0.9478992 , 0.99993288]])

In [50]:
data['IssueDescription'][:3]

0    The spacing on markdown preview has headings (...
1    <!-- ⚠️⚠️ Do Not Delete This! bug_report_templ...
2    <!-- ⚠️⚠️ Do Not Delete This! bug_report_templ...
Name: IssueDescription, dtype: object

In [53]:
data['PrDescription'][0]

'This PR fixes #102036\\r\\n\\r\\n- So the first issue is that most elements have a margin-top and margin-bottom.\\r\\nIt made sense to remove the margin-top and let top elements push others down. (otherwise you fall into margin-collapse hell)\\r\\n- This means the body needs a padding on the top so that any first element has a gap (and not just the H1)\\r\\n- Headings have a smaller bottom margin so as to look more like a section\\r\\n- If 2 paragraphs are next to each other, the second paragraph reduces its top margin to bring them closer together.\\r\\n\\r\\n## Before\\r\\n![image](https://user-images.githubusercontent.com/936006/87351758-ce286c80-c551-11ea-80e5-13af706cd8ac.png)\\r\\n\\r\\n\\r\\n## After\\r\\n![image](https://user-images.githubusercontent.com/936006/87352076-5d358480-c552-11ea-9fa0-c5fcf3861712.png)\\r\\n\\r\\npings @mjbvz \\r\\n\\r\\n\\r\\n'